In [14]:
import os
os.chdir('/content/drive/Shareddrives/yujung kim/Study/Python/time_series')

In [9]:
import gzip
f = gzip.open('data/solar_AL.txt.gz', 'rb')
file_content = f.read()
f.close()

In [10]:
len(file_content)

180018000

In [11]:
len(file_content[0])

TypeError: ignored

In [13]:
file_content[1]

46

# preprocessing

In [43]:
n = 10
P = 5
m = 2
h = 2
dat = torch.rand((10,2))

In [44]:
dat

tensor([[0.9861, 0.6627],
        [0.2118, 0.0978],
        [0.6806, 0.6820],
        [0.7443, 0.9650],
        [0.0309, 0.8681],
        [0.1555, 0.1170],
        [0.9377, 0.4539],
        [0.4180, 0.4556],
        [0.5504, 0.0405],
        [0.4267, 0.9333]])

In [45]:
import torch
import numpy as np

In [49]:
def _batchify(idx_set, h):
    n = len(idx_set)
    X = torch.zeros((n, P, m))
    Y = torch.zeros((n, h, m))
    for i in range(n):
        end = idx_set[i] - h + 1
        start = end - P
        print(start)
        print(end)
        X[i, :, :] = dat[start:end, :]
        # Y[i, :, :] = torch.from_numpy(self.dat[idx_set[i] - self.h:idx_set[i], :])
        Y[i, :, :] = dat[end:(idx_set[i]+1), :]

    return [X, Y]

In [50]:
train_set = range(P + h - 1, 8)

In [51]:
out = _batchify(train_set, h)

0
5
1
6


In [52]:
out[0]

tensor([[[0.9861, 0.6627],
         [0.2118, 0.0978],
         [0.6806, 0.6820],
         [0.7443, 0.9650],
         [0.0309, 0.8681]],

        [[0.2118, 0.0978],
         [0.6806, 0.6820],
         [0.7443, 0.9650],
         [0.0309, 0.8681],
         [0.1555, 0.1170]]])

In [53]:
out[1]

tensor([[[0.1555, 0.1170],
         [0.9377, 0.4539]],

        [[0.9377, 0.4539],
         [0.4180, 0.4556]]])

In [ ]:
def _batchify(self, idx_set, horizon):
    n = len(idx_set)
    X = torch.zeros((n, self.P, self.m))
    Y = torch.zeros((n, self.h, self.m))
    for i in range(n):
        end = idx_set[i] - self.h + 1
        start = end - self.P
        X[i, :, :] = torch.from_numpy(self.dat[start:end, :])
        # Y[i, :, :] = torch.from_numpy(self.dat[idx_set[i] - self.h:idx_set[i], :])
        Y[i, :, :] = torch.from_numpy(self.dat[end:(idx_set[i]+1), :])

# dilation convolution

In [54]:
from torch import nn

In [55]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [67]:
a = torch.rand([3,3,5])

In [72]:
a

tensor([[[0.1479, 0.0011, 0.8917, 0.6447, 0.6145],
         [0.9318, 0.7907, 0.2569, 0.0292, 0.0023],
         [0.2175, 0.6206, 0.8859, 0.3745, 0.3051]],

        [[0.2872, 0.9179, 0.7441, 0.7113, 0.8664],
         [0.3673, 0.8399, 0.3460, 0.3564, 0.8468],
         [0.5236, 0.7244, 0.2439, 0.6985, 0.8026]],

        [[0.5303, 0.5611, 0.3088, 0.2267, 0.9897],
         [0.3585, 0.7831, 0.2847, 0.0463, 0.8943],
         [0.1283, 0.7994, 0.8074, 0.0450, 0.9007]]])

In [68]:
a.stride()

(15, 5, 1)

In [74]:
dilation = Chomp1d(1)
dilation(a)

tensor([[[0.1479, 0.0011, 0.8917, 0.6447],
         [0.9318, 0.7907, 0.2569, 0.0292],
         [0.2175, 0.6206, 0.8859, 0.3745]],

        [[0.2872, 0.9179, 0.7441, 0.7113],
         [0.3673, 0.8399, 0.3460, 0.3564],
         [0.5236, 0.7244, 0.2439, 0.6985]],

        [[0.5303, 0.5611, 0.3088, 0.2267],
         [0.3585, 0.7831, 0.2847, 0.0463],
         [0.1283, 0.7994, 0.8074, 0.0450]]])

In [77]:
dilation = Chomp1d(4)
dilation(a)

tensor([[[0.1479],
         [0.9318],
         [0.2175]],

        [[0.2872],
         [0.3673],
         [0.5236]],

        [[0.5303],
         [0.3585],
         [0.1283]]])

- in_channels: input의 feature dimension
- out_channels: 내가 output으로 내고싶은 dimension
- kernel_size: time step을 얼마만큼 볼 것인가(=frame size = filter size)
- stride: kernel을 얼마만큼씩 이동하면서 적용할 것인가 (Default: 1) -> 아래 추가 설명
- dilation: kernel 내부에서 얼마만큼 띄어서 kernel을 적용할 것인가 (Default: 1) -> 아래 추가 설명
- padding: 한 쪽 방향으로 얼마만큼 padding할 것인가 (그 만큼 양방향으로 적용) (Default: 0)
- groups: kernel의 height를 조절 (Default: 1) -> 아래 추가 설명
- bias: bias term을 둘 것인가 안둘 것인가 (Default: True)
- padding_mode: 'zeros', 'reflect', 'reflect', 'replicate', 'circular' (Default: 'zeros')  

In [78]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


In [81]:
[30]*5

[30, 30, 30, 30, 30]

In [82]:
TCN = TemporalConvNet(num_inputs=3, num_channels=[30]*5)

In [83]:
TCN(a)

tensor([[[8.1061e-01, 2.7302e-01, 2.9360e-01, 4.5863e-01, 3.9243e-01],
         [3.4792e-01, 6.9719e-03, 3.0966e-01, 3.0623e-01, 3.5935e-01],
         [5.4413e-01, 7.7670e-01, 8.5917e-01, 9.9957e-01, 7.5204e-01],
         [7.9278e-01, 7.1448e-01, 6.6490e-01, 6.1590e-01, 8.9166e-01],
         [5.6130e-01, 9.6662e-01, 5.2652e-01, 7.9932e-01, 4.6772e-01],
         [4.2111e-01, 2.3689e-01, 1.6669e-01, 2.5909e-01, 1.9469e-01],
         [2.6186e-01, 3.0454e-01, 3.1718e-01, 2.4048e-01, 3.9153e-01],
         [7.5268e-01, 4.2836e-01, 5.2710e-01, 7.0860e-01, 4.3727e-01],
         [2.5511e-01, 7.3151e-02, 1.0966e-01, 4.3553e-01, 5.6771e-02],
         [1.3422e-01, 1.4912e-01, 3.7952e-01, 2.9615e-01, 3.9074e-01],
         [7.3724e-01, 1.2471e+00, 1.5558e+00, 9.8044e-01, 1.2499e+00],
         [3.1342e-01, 4.9338e-01, 3.0974e-01, 3.0600e-01, 3.3997e-01],
         [6.7936e-02, 3.0101e-01, 3.7523e-01, 2.7707e-01, 4.7376e-01],
         [6.0991e-02, 2.2098e-01, 1.1091e-01, 1.3189e-01, 1.3984e-01],
      

In [85]:
import torch
import torch.nn as nn


x = torch.rand(3,256,16000)
print('input_size:', x.shape)
conv1d = nn.Conv1d(256, 1, kernel_size=3)
print('kernel_size:', conv1d.weight.shape)
out = conv1d(x)
print('output_size:',out.shape)

input_size: torch.Size([3, 256, 16000])
kernel_size: torch.Size([1, 256, 3])
output_size: torch.Size([3, 1, 15998])
